In [17]:
import pandas as py
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import ExtraTreesRegressor
import pickle
df = py.read_csv(r'D:\ML\BTP\preprocessing\final_bowler.csv')

In [18]:
loaded_model = pickle.load(open('et.pkl', 'rb'))

In [19]:
bowlerData=py.read_csv(r'D:\ML\BTP\preprocessing\bowlerData.csv')
bowlerOPP=py.read_csv(r'D:\ML\BTP\preprocessing\bowlerOppositionWiseData.csv')
bowlerVenue=py.read_csv(r'D:\ML\BTP\preprocessing\bowlerVenueWiseData.csv')
venueData=py.read_csv(r'D:\ML\BTP\preprocessing\VenueData.csv')
bowlerVenue


,bowler,venue,Runs_Conceded,Wickets_Taken,balls_bowled,bowlerVenueMatches_Played,five_wickets_haul,three_wickets_haul,four_wickets_haul,maidens,bowlerVenueavg_drm11_score,bowlerVenueEconomy,bowlerVenuestrike_rate
0,A Ashish Reddy,Barabati Stadium,31,1,19,1,0,0,0,0,63.000000,9.789474,19.000
1,A Ashish Reddy,Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...,19,1,19,2,0,0,0,0,31.500000,6.000000,19.000
2,A Ashish Reddy,Eden Gardens,15,1,12,1,0,0,0,0,47.000000,7.500000,12.000
3,A Ashish Reddy,M Chinnaswamy Stadium,83,2,48,4,0,0,0,0,32.500000,10.375000,24.000
4,A Ashish Reddy,MA Chidambaram Stadium,29,1,18,2,0,0,0,0,27.500000,9.666667,18.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4287,Z Khan,Sharjah Cricket Stadium,17,0,17,1,0,0,0,0,34.000000,6.000000,18.000
4288,Z Khan,Sheikh Zayed Stadium,20,1,27,1,0,0,0,1,97.000000,4.444444,27.000
4289,Z Khan,St George's Park,14,1,18,1,0,0,0,0,67.000000,4.666667,18.000
4290,Z Khan,Subrata Roy Sahara Stadium,21,2,24,1,0,0,0,0,94.000000,5.250000,12.000


In [20]:
with open('bowler_encoder.pkl', 'rb') as file:
    bw = pickle.load(file)
with open('venue_encoder.pkl', 'rb') as file:
    vn = pickle.load(file)
with open('batting_team_encoder.pkl', 'rb') as file:
    bt = pickle.load(file)

In [21]:
vn.classes_

array(['Arun Jaitley Stadium', 'Barabati Stadium',
       'Barsapara Cricket Stadium',
       'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium',
       'Brabourne Stadium', 'Buffalo Park', 'De Beers Diamond Oval',
       'Dr DY Patil Sports Academy',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Dubai International Cricket Stadium', 'Eden Gardens',
       'Green Park', 'Himachal Pradesh Cricket Association Stadium',
       'Holkar Cricket Stadium', 'JSCA International Stadium Complex',
       'Kingsmead', 'M Chinnaswamy Stadium', 'MA Chidambaram Stadium',
       'Maharashtra Cricket Association Stadium', 'Narendra Modi Stadium',
       'Nehru Stadium', 'New Wanderers Stadium', 'Newlands',
       'OUTsurance Oval', 'Punjab Cricket Association Stadium',
       'Rajiv Gandhi International Stadium',
       'Saurashtra Cricket Association Stadium', 'Sawai Mansingh Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'Sharjah Cricke

In [22]:
bowler='Mohammed Shami'
venue='Narendra Modi Stadium'
oppposition='Chennai Super Kings'


def bowler_input(bowler,venue,oppposition):

    Matches_Played=bowlerData[bowlerData['bowler']==bowler]['Matches_Played'].values[0]
    avg_drm11_score=bowlerData[bowlerData['bowler']==bowler]['avg_drm11_score'].values[0]
    Economy=bowlerData[bowlerData['bowler']==bowler]['Economy'].values[0]
    strike_rate=bowlerData[bowlerData['bowler']==bowler]['strike_rate'].values[0]


    bowlerOppositionWiseMatches_Played=0
    bowlerOppositionWiseavg_drm11_score=0
    bowlerOppositionWiseEconomy=0
    bowlerOppositionWisestrike_rate=0
    if(bowlerOPP[(bowlerOPP['bowler']==bowler) & (bowlerOPP['batting_team']==oppposition)].empty):  
        bowlerOppositionWiseMatches_Played=bowlerOPP.groupby(['batting_team'])['bowlerOppositionWiseMatches_Played'].mean()[oppposition]
        bowlerOppositionWiseavg_drm11_score=bowlerOPP.groupby(['batting_team'])['bowlerOppositionWiseavg_drm11_score'].mean()[oppposition]
        bowlerOppositionWiseEconomy=bowlerOPP.groupby(['batting_team'])['bowlerOppositionWiseEconomy'].mean()[oppposition]
        bowlerOppositionWisestrike_rate=bowlerOPP.groupby(['batting_team'])['bowlerOppositionWisestrike_rate'].mean()[oppposition]
    else:
        bowlerOppositionWiseMatches_Played=bowlerOPP[(bowlerOPP['bowler']==bowler) & (bowlerOPP['batting_team']==oppposition)]['bowlerOppositionWiseMatches_Played'].values[0]
        bowlerOppositionWiseavg_drm11_score=bowlerOPP[(bowlerOPP['bowler']==bowler) & (bowlerOPP['batting_team']==oppposition)]['bowlerOppositionWiseavg_drm11_score'].values[0]
        bowlerOppositionWiseEconomy=bowlerOPP[(bowlerOPP['bowler']==bowler) & (bowlerOPP['batting_team']==oppposition)]['bowlerOppositionWiseEconomy'].values[0]
        bowlerOppositionWisestrike_rate=bowlerOPP[(bowlerOPP['bowler']==bowler) & (bowlerOPP['batting_team']==oppposition)]['bowlerOppositionWisestrike_rate'].values[0]        
    bowlerVenueMatches_Played=0
    bowlerVenueavg_drm11_score=0
    bowlerVenueEconomy=0
    bowlerVenuestrike_rate=0
    if(bowlerVenue[(bowlerVenue['bowler']==bowler) & (bowlerVenue['venue']==venue)].empty):
        bowlerVenueMatches_Played=bowlerVenue.groupby(['venue'])['bowlerVenueMatches_Played'].mean()[venue]
        bowlerVenueavg_drm11_score=bowlerVenue.groupby(['venue'])['bowlerVenueavg_drm11_score'].mean()[venue]
        bowlerVenueEconomy=bowlerVenue.groupby(['venue'])['bowlerVenueEconomy'].mean()[venue]
        bowlerVenuestrike_rate=bowlerVenue.groupby(['venue'])['bowlerVenuestrike_rate'].mean()[venue]
    else:
        bowlerVenueMatches_Played=bowlerVenue[(bowlerVenue['bowler']==bowler) & (bowlerVenue['venue']==venue)]['bowlerVenueMatches_Played'].values[0]
        bowlerVenueavg_drm11_score=bowlerVenue[(bowlerVenue['bowler']==bowler) & (bowlerVenue['venue']==venue)]['bowlerVenueavg_drm11_score'].values[0]
        bowlerVenueEconomy=bowlerVenue[(bowlerVenue['bowler']==bowler) & (bowlerVenue['venue']==venue)]['bowlerVenueEconomy'].values[0]
        bowlerVenuestrike_rate=bowlerVenue[(bowlerVenue['bowler']==bowler) & (bowlerVenue['venue']==venue)]['bowlerVenuestrike_rate'].values[0]
    Venue_Matches_Played = venueData[venueData['venue']
                                        == venue]['Venue_Matches_Played'].values[0]
    Venue_Avg_Batsmen_Strike_Rate = venueData[venueData['venue']
                                                == venue]['Venue_Avg_Batsmen_Strike_Rate'].values[0]
    Venue_Avg_Batsmen_Average = venueData[venueData['venue']
                                            == venue]['Venue_Avg_Batsmen_Average'].values[0]
    Venue_Avg_Bowler_Economy = venueData[venueData['venue']
                                            == venue]['Venue_Avg_Bowler_Economy'].values[0]
    Venue_Avg_Bowler_Average = venueData[venueData['venue']
                                            == venue]['Venue_Avg_Bowler_Average'].values[0]
    Venue_dream11 = venueData[venueData['venue']
                                == venue]['Venue_dream11'].values[0]
    
    bow = bw.transform([bowler])
    ven = vn.transform([venue])
    oppo = bt.transform([oppposition])
    data = np.array([[bow[0], ven[0], oppo[0], Matches_Played, avg_drm11_score,Economy,	strike_rate,	bowlerOppositionWiseMatches_Played, bowlerOppositionWiseavg_drm11_score,	bowlerOppositionWiseEconomy, bowlerOppositionWisestrike_rate,	bowlerVenueMatches_Played,	bowlerVenueavg_drm11_score,	bowlerVenueEconomy,
                    bowlerVenuestrike_rate,Venue_Matches_Played,	Venue_Avg_Batsmen_Strike_Rate,	Venue_Avg_Batsmen_Average,	Venue_Avg_Bowler_Economy,	Venue_Avg_Bowler_Average,	Venue_dream11]])
    return data

In [23]:
bowlerOppositionWiseavg_drm11_score=bowlerOPP[(bowlerOPP['bowler']==bowler) & (bowlerOPP['batting_team']==oppposition)]['bowlerOppositionWiseavg_drm11_score'].values[0]
bowlerOppositionWiseavg_drm11_score=bowlerOPP[(bowlerOPP['bowler']==bowler) & (bowlerOPP['batting_team']==oppposition)]['bowlerOppositionWiseavg_drm11_score']
bowlerOppositionWiseavg_drm11_score

1977    73.0
Name: bowlerOppositionWiseavg_drm11_score, dtype: float64

In [24]:
bow = bw.transform([bowler])
ven = vn.transform([venue])
oppo = bt.transform([oppposition])

In [25]:
a=bowler_input(bowler,venue,oppposition)

In [26]:
a

array([[290.        ,  19.        ,   0.        , 110.        ,
         80.61818182,   7.82070607,  17.50694444,  14.        ,
         73.        ,   7.02484472,  23.        ,  13.        ,
         91.30769231,   7.21212121,  14.14285714,  28.        ,
        130.62620854,   1.30626209,   7.83757251,   1.30626209,
        326.35714286]])

In [27]:
p=loaded_model.predict(a)

C:\Users\Aryan shukla\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but ExtraTreesRegressor was fitted with feature names
  warnings.warn(


In [28]:
p

array([83.58705151])

In [29]:
df[df['bowler']=='H Sharma']

,bowler,venue,batting_team,dream 11 score,Matches_Played,avg_drm11_score,Economy,strike_rate,bowlerOppositionWiseMatches_Played,bowlerOppositionWiseavg_drm11_score,...,bowlerVenueMatches_Played,bowlerVenueavg_drm11_score,bowlerVenueEconomy,bowlerVenuestrike_rate,Venue_Matches_Played,Venue_Avg_Batsmen_Strike_Rate,Venue_Avg_Batsmen_Average,Venue_Avg_Bowler_Economy,Venue_Avg_Bowler_Average,Venue_dream11
12065,H Sharma,M Chinnaswamy Stadium,Gujarat Titans,36,1,36.0,9.333333,19.0,1,36.0,...,1,36.0,9.333333,19.0,87,132.025235,1.320252,7.921514,1.320252,325.16092


In [30]:
bow = bw.transform(['H Sharma'])